In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [3]:
loader1 = TextLoader("./data/nlp-keywords.txt", encoding='utf-8')
loader2 = TextLoader("./data/book_document.txt", encoding='utf-8')

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

In [5]:
# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

In [7]:
DB_PATH = "./chroma_db"     # 저장 경로

# DB 생성
db = Chroma.from_documents(
    documents=split_doc1 + split_doc2,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    persist_directory=DB_PATH,
    collection_name='db_2'
)

In [ ]:
retriever = db.as_retriever()       # 벡터 저장소를 기반으로 VectorStoreRetriever를 생성

In [10]:
# 기본 값으로 설정된 4개 문서를 유사도 검색을 수행하여 조회합니다.
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이

In [11]:
# 다양성이 높은 더 많은 문서 검색
# . k: 반환할 문서 수 (기본값: 4)
# . fetch_k: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
# . lambda_mult: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5, 0: 유사도 점수만 고려, 1: 다양성만 고려)

In [12]:
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25, "fetch_k": 10}
)

In [13]:
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이

In [14]:
# MMR 알고리즘을 위해 더 많은 문서를 가져오되 상위 2개만 반환
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10})

In [15]:
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정

In [26]:
# 특정 임계값 이상의 유사도를 가진 문서만 검색
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
)

In [27]:
retriever.invoke("Word2Vec 에 대하여 알려줘")

No relevant docs were retrieved using the relevance score threshold 0.8


[]

In [28]:
# 가장 유사한 단일 문서만 검색
retriever = db.as_retriever(search_kwargs={"k": 1})

In [31]:
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source')]

In [38]:
# 특정 메타데이터 필터 적용
retriever = db.as_retriever(
    search_kwargs={"filter": {"source": "./data/book_document.txt"}, "k": 2}
)

In [39]:
retriever.invoke("숫자는 거짓말을 한다. 에 대해서 알려줘")

[Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 숫자는 거짓말을 한다\n- 저자 : 알베르토 카이로\n- 발행처 : 웅진지식하우스 20201013\n- isbn : 9788901245591\n- 청구기호 : 일 310.1-알848ㅅ\n- 등록번호 : CLP000184432\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 피노자의 형이상학\n- 저자 : 김은주\n- 발행처 : 민음사 20240503\n- isbn : 9788937446009\n- 청구기호 : 일 166.2-김68ㅅ\n- 등록번호 : CLP000299472\n- 소장위치 : 첫 번째 책장 첫 번째 열\n\n\n### 제목 : 파이썬 머신러닝 판다스 데이터 분석 (개정판)\n- 저자 : 오승환\n- 발행처 : 정보문화사 20240625\n- isbn : 9788956749808\n- 청구기호 : 일 001.12-오98o\n- 등록번호 : CLP000178599\n- 소장위치 : 두 번재 책장 첫 번째 열\n\n\n### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열'),
 Document(metadata={'source': './data/book_document.txt'}, page_content='### 제목 : 손에잡히는 파이썬\n- 저자 : 문용준\n- 발행처 : BJpublic(비제이퍼블릭) 2018\n- isbn : 9791186697726\n- 청구기호 : 일 005.133-문66ㅅ\n- 등록번호 : CLP000178588\n- 소장위치 : 세 번째 책장 두 번 째 열\n\n\n### 제목 : 작심 3일 파이썬 Python\n- 저자 : 황덕창\n- 발행처 : 스포트라잇북